# 카카오 OpenAPI 연동 (이미지 검색)

## #01. 준비과정

### [1] 패키지 참조

In [3]:
import requests
import json
import datetime as dt
import os
import concurrent.futures as futures

### [2] 접속할 데이터의 URL

In [4]:
urlFmt = "https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}"

### [3] 요청 변수

In [6]:
query = "파이썬"
page = 1
size = 50
key = "dad57e5d6bd34ed6ab195acb162c5995"

## #03. 데이터 요청하기

### [1] 세션 생성

In [8]:
session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Authorization": "KakaoAK %s" % key
})

### [2] 데이터 가져오기

마지막 페이지임을 의미하는 meta > is_end 라는 값이 True로 반환될 때 까지 page 변수를 1씩 증가시키면서 반복 수행

In [9]:
page = 1
isEnd = False

# 수집된 데이터를 모아 놓을 빈 리스트 (합계 구하는 것과 비슷함)
mydata = []
while not isEnd:
    url = urlFmt.format(query=query, page=page, size=size)
    print(url)

    try:
        r = session.get(url)

        if r.status_code != 200:
            msg = "[%d error] %s 에러가 발생함" % (r.status_cod, r.reason)
            raise Exception(msg)
    
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

        # 에러 발생시 나머지 페이지 중단
        # break

        # 에러 발생시 다음 페이지 시도
        continue

    # 응답 결과 확인
    r.encoding = "utf-8"
    mydict = json.loads(r.text)

    # 수집 결과를 미리 준비한 빈 리스트에 추가
    mydata += mydict['documents']
    print("%d 페이지로부터 %d건의 데이터를 수집했습니다." % (page, len(mydict['documents'])))

    # 마지막 페이지인지 확인
    isEnd = mydict['meta']['is_end']

    # 페이지 번호를 1 증가
    page += 1

print("fin :)")


https://dapi.kakao.com/v2/search/image?query=파이썬&page=1&size=50
1 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=2&size=50
2 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=3&size=50
3 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=4&size=50
4 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=5&size=50
5 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=6&size=50
6 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=7&size=50
7 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=8&size=50
8 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=9&size=50
9 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=10&size=50
10 페이지로부터 50건의 데이터를 수집했습니다.
https://dapi.kakao.com/v2/search/image?query=파이썬&page=11&size=50
11 페이지로부터 50건의 데이터를 수집했

## #04. 데이터 활용
### [1] 수집 결과를 csv로 저장하기

In [13]:
with open("%s_이미지_검색결과.csv" % query, "w", encoding= "utf-8") as f:
    f.write("컬렉션, 출처,가로,세로, 문서URL, 미리보기URL, 이미지URL\n")

    for i, v in enumerate(mydata):
        collection, thumbnail_url, image_url, width, height, display_sitename, doc_url, datetime = v.values()
        f.write("%s,%s,%s,%s,%s,%s,%s\n" % (collection, display_sitename, width, height, doc_url, thumbnail_url, image_url))

print("fin :)")


fin :)


### [2] 이미지 다운로드
#### (1) 전체 데이터 수 확인

In [14]:
print("다운로드 받을 이미지 수: %d" % len(mydata))

다운로드 받을 이미지 수: 2038


#### (2) 다운로드 받을 폴더 생성

In [15]:
downloadDir = "%s_%s" % (query, dt.datetime.now().strftime("%y%m%d_%H%M%S"))
print (downloadDir)

if not os.path.exists(downloadDir):
    os.mkdir(downloadDir)

파이썬_231211_114326


#### (3) 이미지 다운로드 함수 구현
동기 처리를 위해서는 함수를 반드시 정의해야 하는 것은 아니지만, 비동기 처리를 염두한다면 단위 기능을 함수로 묶어야 한다.

In [20]:
def kakaoImageDownload(session, docUrl, imageUrl, savePath):
    # 이미지 입장에서는 자신이 포함된 페이지(docUrl)가 이전 접속 페이지이므로 referer 설정을 변경한다
    session.headers.update({"Referer": docUrl})
    
    try:
        r = session.get(imageUrl, stream = True)
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" %(r.status_code, r.reason)
            raise Exception(msg)

    except Exception as e:
        print("접속에 실패했습니다")
        print(e)

    # 인코딩 형식 지정
    r.encoding = "utf-8"
    
    # 수신 결과를 파일로 저장
    with open(savePath, "wb") as f:
        f.write(r.raw.read())
        print("%s 파일이 저장되었습니다." % savePath)
        

#### (4) 동기식 호출

작업을 하나씩 순차적으로 수행한다.

수행 시간은 약 4분 정도 걸림

In [22]:
for i, v in enumerate(mydata):
    kakaoImageDownload(session, v['doc_url'], v['image_url'], "%s/%05d.png" % (downloadDir, i))

파이썬_231211_114326/00000.png 파일이 저장되었습니다.
파이썬_231211_114326/00001.png 파일이 저장되었습니다.
파이썬_231211_114326/00002.png 파일이 저장되었습니다.
파이썬_231211_114326/00003.png 파일이 저장되었습니다.
파이썬_231211_114326/00004.png 파일이 저장되었습니다.
파이썬_231211_114326/00005.png 파일이 저장되었습니다.
파이썬_231211_114326/00006.png 파일이 저장되었습니다.
파이썬_231211_114326/00007.png 파일이 저장되었습니다.
파이썬_231211_114326/00008.png 파일이 저장되었습니다.
파이썬_231211_114326/00009.png 파일이 저장되었습니다.
파이썬_231211_114326/00010.png 파일이 저장되었습니다.
파이썬_231211_114326/00011.png 파일이 저장되었습니다.
파이썬_231211_114326/00012.png 파일이 저장되었습니다.
파이썬_231211_114326/00013.png 파일이 저장되었습니다.
파이썬_231211_114326/00014.png 파일이 저장되었습니다.
파이썬_231211_114326/00015.png 파일이 저장되었습니다.
파이썬_231211_114326/00016.png 파일이 저장되었습니다.
파이썬_231211_114326/00017.png 파일이 저장되었습니다.
파이썬_231211_114326/00018.png 파일이 저장되었습니다.
파이썬_231211_114326/00019.png 파일이 저장되었습니다.
파이썬_231211_114326/00020.png 파일이 저장되었습니다.
파이썬_231211_114326/00021.png 파일이 저장되었습니다.
파이썬_231211_114326/00022.png 파일이 저장되었습니다.
파이썬_231211_114326/00023.png 파일이 저장되었습니다.
파이썬_231211_11432

#### (5) 비동기식 호출

In [ ]:
with futures.ThreadPoolExecutor(max_workers: 100) as executor:
    for i, v in enumerate(mydata):
        executor.submit(kakaoImageDownload, session, v['doc_url'], v[image_url], "%s/%05d.png") % (downloadDir, i)